In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
k = 4
tree_depth = 8
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [13]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [14]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [15]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [16]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 2.0072975158691406 | KNN Loss: 6.227202892303467 | BCE Loss: 2.0072975158691406
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 1.9876428842544556 | KNN Loss: 6.2271728515625 | BCE Loss: 1.9876428842544556
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 1.9654991626739502 | KNN Loss: 6.227078914642334 | BCE Loss: 1.9654991626739502
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 2.007986545562744 | KNN Loss: 6.227046489715576 | BCE Loss: 2.007986545562744
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 1.93807053565979 | KNN Loss: 6.227117538452148 | BCE Loss: 1.93807053565979
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 1.890448808670044 | KNN Loss: 6.2272539138793945 | BCE Loss: 1.890448808670044
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 1.8803540468215942 | KNN Loss: 6.227087020874023 | BCE Loss: 1.8803540468215942
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 1.8928595781326294 | KNN Loss: 6.227339267730713 | BCE Loss: 1.892859

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 1.1139508485794067 | KNN Loss: 6.227087020874023 | BCE Loss: 1.1139508485794067
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 1.1409372091293335 | KNN Loss: 6.227234840393066 | BCE Loss: 1.1409372091293335
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 1.1305830478668213 | KNN Loss: 6.227318286895752 | BCE Loss: 1.1305830478668213
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 1.154988408088684 | KNN Loss: 6.227245807647705 | BCE Loss: 1.154988408088684
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 1.1592546701431274 | KNN Loss: 6.227466106414795 | BCE Loss: 1.1592546701431274
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 1.1000138521194458 | KNN Loss: 6.227219104766846 | BCE Loss: 1.1000138521194458
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 1.1004137992858887 | KNN Loss: 6.227105617523193 | BCE Loss: 1.1004137992858887
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 1.113079309463501 | KNN Loss: 6.2272515296936035 | BC

Epoch 21 / 500 | iteration 15 / 30 | Total Loss: 1.0572924613952637 | KNN Loss: 6.226929664611816 | BCE Loss: 1.0572924613952637
Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 1.0835916996002197 | KNN Loss: 6.226919174194336 | BCE Loss: 1.0835916996002197
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 1.0618128776550293 | KNN Loss: 6.227316379547119 | BCE Loss: 1.0618128776550293
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 1.0492719411849976 | KNN Loss: 6.227105617523193 | BCE Loss: 1.0492719411849976
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 1.0456345081329346 | KNN Loss: 6.227390289306641 | BCE Loss: 1.0456345081329346
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 1.0723578929901123 | KNN Loss: 6.227113723754883 | BCE Loss: 1.0723578929901123
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 1.0711735486984253 | KNN Loss: 6.227255821228027 | BCE Loss: 1.0711735486984253
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 1.0376567840576172 | KNN Loss: 6.227641582489014 |

Epoch 32 / 500 | iteration 5 / 30 | Total Loss: 1.0753090381622314 | KNN Loss: 6.227157115936279 | BCE Loss: 1.0753090381622314
Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 1.029982089996338 | KNN Loss: 6.227000713348389 | BCE Loss: 1.029982089996338
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 1.0515832901000977 | KNN Loss: 6.227030277252197 | BCE Loss: 1.0515832901000977
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 1.0583832263946533 | KNN Loss: 6.2273454666137695 | BCE Loss: 1.0583832263946533
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 1.0431360006332397 | KNN Loss: 6.22691535949707 | BCE Loss: 1.0431360006332397
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 1.018707513809204 | KNN Loss: 6.227166652679443 | BCE Loss: 1.018707513809204
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 1.0483719110488892 | KNN Loss: 6.227583885192871 | BCE Loss: 1.0483719110488892
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 1.065049171447754 | KNN Loss: 6.227123737335205 | BCE L

Epoch 42 / 500 | iteration 25 / 30 | Total Loss: 1.0463571548461914 | KNN Loss: 6.22697114944458 | BCE Loss: 1.0463571548461914
Epoch 43 / 500 | iteration 0 / 30 | Total Loss: 1.0512670278549194 | KNN Loss: 6.227015495300293 | BCE Loss: 1.0512670278549194
Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 1.04410719871521 | KNN Loss: 6.226932048797607 | BCE Loss: 1.04410719871521
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 1.048304557800293 | KNN Loss: 6.226841449737549 | BCE Loss: 1.048304557800293
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 1.0363131761550903 | KNN Loss: 6.22735071182251 | BCE Loss: 1.0363131761550903
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 1.0411527156829834 | KNN Loss: 6.226919174194336 | BCE Loss: 1.0411527156829834
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 1.091540813446045 | KNN Loss: 6.227146625518799 | BCE Loss: 1.091540813446045
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 1.0506095886230469 | KNN Loss: 6.22714900970459 | BCE Loss: 1

...

Epoch 455 / 500 | iteration 20 / 30 | Total Loss: 1.0498290061950684 | KNN Loss: 6.226836204528809 | BCE Loss: 1.0498290061950684
Epoch 455 / 500 | iteration 25 / 30 | Total Loss: 1.0303428173065186 | KNN Loss: 6.227087497711182 | BCE Loss: 1.0303428173065186
Epoch 456 / 500 | iteration 0 / 30 | Total Loss: 1.0508776903152466 | KNN Loss: 6.226822376251221 | BCE Loss: 1.0508776903152466
Epoch 456 / 500 | iteration 5 / 30 | Total Loss: 1.0384671688079834 | KNN Loss: 6.2269062995910645 | BCE Loss: 1.0384671688079834
Epoch 456 / 500 | iteration 10 / 30 | Total Loss: 1.049192190170288 | KNN Loss: 6.226996421813965 | BCE Loss: 1.049192190170288
Epoch 456 / 500 | iteration 15 / 30 | Total Loss: 1.0369064807891846 | KNN Loss: 6.22691535949707 | BCE Loss: 1.0369064807891846
Epoch 456 / 500 | iteration 20 / 30 | Total Loss: 1.0283255577087402 | KNN Loss: 6.226684093475342 | BCE Loss: 1.0283255577087402
Epoch 456 / 500 | iteration 25 / 30 | Total Loss: 1.0855860710144043 | KNN Loss: 6.22651338577

Epoch 466 / 500 | iteration 10 / 30 | Total Loss: 1.0367319583892822 | KNN Loss: 6.226821422576904 | BCE Loss: 1.0367319583892822
Epoch 466 / 500 | iteration 15 / 30 | Total Loss: 1.0570251941680908 | KNN Loss: 6.2269287109375 | BCE Loss: 1.0570251941680908
Epoch 466 / 500 | iteration 20 / 30 | Total Loss: 1.0570476055145264 | KNN Loss: 6.227108001708984 | BCE Loss: 1.0570476055145264
Epoch 466 / 500 | iteration 25 / 30 | Total Loss: 1.0634664297103882 | KNN Loss: 6.22666072845459 | BCE Loss: 1.0634664297103882
Epoch 467 / 500 | iteration 0 / 30 | Total Loss: 1.037036418914795 | KNN Loss: 6.226956844329834 | BCE Loss: 1.037036418914795
Epoch 467 / 500 | iteration 5 / 30 | Total Loss: 1.06328547000885 | KNN Loss: 6.227017879486084 | BCE Loss: 1.06328547000885
Epoch 467 / 500 | iteration 10 / 30 | Total Loss: 1.060294270515442 | KNN Loss: 6.226694107055664 | BCE Loss: 1.060294270515442
Epoch 467 / 500 | iteration 15 / 30 | Total Loss: 1.04581618309021 | KNN Loss: 6.226778984069824 | BCE 

Epoch 477 / 500 | iteration 0 / 30 | Total Loss: 1.0306501388549805 | KNN Loss: 6.22696590423584 | BCE Loss: 1.0306501388549805
Epoch 477 / 500 | iteration 5 / 30 | Total Loss: 1.0561633110046387 | KNN Loss: 6.226858615875244 | BCE Loss: 1.0561633110046387
Epoch 477 / 500 | iteration 10 / 30 | Total Loss: 1.0134172439575195 | KNN Loss: 6.226982116699219 | BCE Loss: 1.0134172439575195
Epoch 477 / 500 | iteration 15 / 30 | Total Loss: 1.048302173614502 | KNN Loss: 6.2264885902404785 | BCE Loss: 1.048302173614502
Epoch 477 / 500 | iteration 20 / 30 | Total Loss: 1.075028419494629 | KNN Loss: 6.227017402648926 | BCE Loss: 1.075028419494629
Epoch 477 / 500 | iteration 25 / 30 | Total Loss: 1.039601445198059 | KNN Loss: 6.227046012878418 | BCE Loss: 1.039601445198059
Epoch 478 / 500 | iteration 0 / 30 | Total Loss: 1.0403178930282593 | KNN Loss: 6.226950168609619 | BCE Loss: 1.0403178930282593
Epoch 478 / 500 | iteration 5 / 30 | Total Loss: 1.065673589706421 | KNN Loss: 6.226911544799805 | 

Epoch 487 / 500 | iteration 20 / 30 | Total Loss: 1.0551360845565796 | KNN Loss: 6.226767539978027 | BCE Loss: 1.0551360845565796
Epoch 487 / 500 | iteration 25 / 30 | Total Loss: 1.0185307264328003 | KNN Loss: 6.226561546325684 | BCE Loss: 1.0185307264328003
Epoch 488 / 500 | iteration 0 / 30 | Total Loss: 1.0295162200927734 | KNN Loss: 6.226572513580322 | BCE Loss: 1.0295162200927734
Epoch 488 / 500 | iteration 5 / 30 | Total Loss: 1.0006952285766602 | KNN Loss: 6.226861953735352 | BCE Loss: 1.0006952285766602
Epoch 488 / 500 | iteration 10 / 30 | Total Loss: 1.0619583129882812 | KNN Loss: 6.226715564727783 | BCE Loss: 1.0619583129882812
Epoch 488 / 500 | iteration 15 / 30 | Total Loss: 1.0766658782958984 | KNN Loss: 6.226724147796631 | BCE Loss: 1.0766658782958984
Epoch 488 / 500 | iteration 20 / 30 | Total Loss: 1.0658035278320312 | KNN Loss: 6.226887226104736 | BCE Loss: 1.0658035278320312
Epoch 488 / 500 | iteration 25 / 30 | Total Loss: 1.0724034309387207 | KNN Loss: 6.226710319

Epoch 498 / 500 | iteration 10 / 30 | Total Loss: 1.0423955917358398 | KNN Loss: 6.226851463317871 | BCE Loss: 1.0423955917358398
Epoch 498 / 500 | iteration 15 / 30 | Total Loss: 1.053356647491455 | KNN Loss: 6.227038860321045 | BCE Loss: 1.053356647491455
Epoch 498 / 500 | iteration 20 / 30 | Total Loss: 1.0328301191329956 | KNN Loss: 6.2270331382751465 | BCE Loss: 1.0328301191329956
Epoch 498 / 500 | iteration 25 / 30 | Total Loss: 1.0257878303527832 | KNN Loss: 6.226958274841309 | BCE Loss: 1.0257878303527832
Epoch 499 / 500 | iteration 0 / 30 | Total Loss: 1.0898840427398682 | KNN Loss: 6.227123737335205 | BCE Loss: 1.0898840427398682
Epoch 499 / 500 | iteration 5 / 30 | Total Loss: 1.0536696910858154 | KNN Loss: 6.226707935333252 | BCE Loss: 1.0536696910858154
Epoch 499 / 500 | iteration 10 / 30 | Total Loss: 1.0438902378082275 | KNN Loss: 6.226748466491699 | BCE Loss: 1.0438902378082275
Epoch 499 / 500 | iteration 15 / 30 | Total Loss: 1.0359838008880615 | KNN Loss: 6.2267909049

In [17]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 2.9260,  3.8751,  2.5815,  3.5753,  3.4575,  0.7073,  2.6673,  2.1985,
          2.3084,  1.9959,  2.2377,  2.2015,  0.7877,  1.8225,  1.2886,  1.5240,
          2.8093,  3.1821,  2.8012,  2.3064,  1.7457,  2.9548,  2.2895,  2.6397,
          2.5373,  1.7417,  2.1251,  1.4121,  1.4929,  0.3215, -0.2411,  0.9961,
          0.2093,  0.9268,  1.5295,  1.4739,  1.0045,  3.3175,  0.8013,  1.3213,
          0.9638, -0.7019, -0.2381,  2.3380,  2.1909,  0.7373, -0.1839,  0.0959,
          1.4636,  2.4984,  1.8227,  0.1449,  1.4272,  0.5246, -0.6349,  1.1086,
          1.4808,  1.3742,  1.3425,  1.8266,  0.5733,  0.8346,  0.1406,  1.7248,
          1.3218,  1.6694, -1.8264,  0.3082,  2.2918,  2.1461,  2.5514,  0.4280,
          1.3545,  2.4628,  2.0004,  1.2954,  0.2204,  0.7366,  0.2152,  1.5899,
          0.0288,  0.3820,  1.8390, -0.3779,  0.2434, -1.0678, -2.2868, -0.2612,
          0.5480, -1.8348,  0.4684, -0.1306, -0.5724, -0.9397,  0.5634,  1.2697,
         -0.6998, -0.6983,  

In [18]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [21]:
dataset_ = [d[0].cpu() for d in dataset]

In [22]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 78.51it/s]


In [23]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [45]:
clusters = DBSCAN(eps=0.01, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [46]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [48]:
tensor_dataset = torch.stack(dataset_)

In [49]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [50]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [51]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [52]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [53]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [54]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [55]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [56]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [57]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [58]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [59]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [60]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [61]:
losses = []
accs = []
sparsity = []

In [62]:
set(clusters)

{-1, 0}

In [42]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
Epoch: 00 | Batch: 000 / 030 | Total loss: 9.569 | Reg loss: 0.009 | Tree loss: 9.569 | Accuracy: 0.000000 | 0.307 sec/iter
Epoch: 00 | Batch: 001 / 030 | Total loss: 9.549 | Reg loss: 0.009 | Tree loss: 9.549 | Accuracy: 0.000000 | 0.278 sec/iter
Epoch: 00 | Batch: 002 / 030 | Total loss: 9.530 | Reg loss: 0.009 | Tree loss: 9.530 | Accuracy: 0.000000 | 0.263 sec/iter
Epoch: 00 | Batch: 003 / 030 | Total loss: 9.511 | Reg loss: 0.008 | Tree loss: 9.511 | Accuracy: 0.000000 | 0.255 sec/iter
Epoch: 00 | Batch: 004 / 030 | Total loss: 9.492 | Reg loss: 0.008 | Tree loss: 9.492 | Accuracy: 0.000000 | 0.251 sec/iter
Epoch: 00 | Batch: 005 / 030 | Total loss: 9.472 | Reg loss: 0.008 | Tree loss: 9.472 | Accuracy: 0.000000 | 0.248 sec/iter
Epoch: 00 | Batch: 006 / 030 | Total loss: 9.450 | Reg loss: 0.008 | Tree loss: 9.450 | Accuracy: 0.021484 | 0.249 sec/iter
Epoch: 00 | Batch

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    for cond in conds:
        cond.weights = cond.weights / normalizers
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")